# ALeRCE Stamp Classifier notebook

```Author: Rodrigo Carrasco-Davis, modifications: Ignacio Reyes, Francisco Förster. Last updated: 20221104```

For more information about the ALeRCE broker, please visit http://alerce.science/, or take a look to our presentation paper: F. Förster et al 2021 AJ 161 242 (https://doi.org/10.3847/1538-3881/abe9bc).

This notebook uses the ALeRCE API through our Python client, which can be installed with `pip install alerce`. To get the newest version you can clone and install https://github.com/alercebroker/alerce_client (use the command `python -m pip install -e .` to install). The documentation is available at https://alerce.readthedocs.io/en/latest/index.html

We recommend that you run this tutorial notebook from colab in this [link](https://colab.research.google.com/github/alercebroker/usecases/blob/master/notebooks/ADASS_XXXII_Nov2022_tutorial/ALeRCE_ML_Stamp_Classifier.ipynb).

# Table of contents:
* [Introduction](#intro)
    * [Training data](#training_data) 
    * [Convolutional neural network architecture](#CNN)
    * [Classification results](#Results)
* [ALeRCE frontend and Supernova Hunter](#Front)
* [Requirements](#req)
* [ALeRCE client](#client)
    * [Plotting light curves](#plot_lc)
    * [Retrieve classified alerts](#get_alerts)
    * [Plot some examples](#examples)
    * [Get a sequence of stamps](#stamp_series)
    * [Get spatial distribution](#spatial_distribution)

# Introduction <a class="anchor" id="intro"></a>

In this notebook, we will explore the results of the ALeRCE stamp classifier, explained in **R. Carrasco-Davis et al 2021 AJ 162 231 (https://doi.org/10.3847/1538-3881/ac0ef1)**, which is based on a convolutional neural network (CNN) that uses the images and metadata of the first alert only as input. The stamp classifier uses the first alert to quickly discriminate between active galactic nuclei (AGN), supernovae(SNe), variable stars (VS), asteroids, and bogus alerts. The predictions of the classifier are used to sort the alerts by their probability of being a supernova so an astronomer can see them in the [Supernova Hunter](https://snhunter.alerce.online/) web interface, to then report the candidates to TNS. We also have a light curve based classifier **(P. Sánchez-Sáez et al 2021 AJ 161 141; https://doi.org/10.3847/1538-3881/abd5c1)**, which classifies objects based on their light curve. The purpose of this classifier is to provide a more refined classification starting with at least 6 detections in a given band. For more information please check the AGN, Variable S

### Training Data: <a class="anchor" id="training_data"></a>
Using the labeled set from P. Sánchez-Sáez et al 2021 (The ALeRCE light curve classifier paper), we gathered 52,244 first alerts to build our training set. The number of samples of AGN, SN, VS, asteroid, and bogus are 14,966(29%), 1620 (3%), 14,996 (29%), 9899 (19%), and 10,763(20%) respectively. The images in the alert are the science, template and difference image. We noticed each of these classes can be classified fast since they present specific properties that can be found in the information of the first alert:

+ AGN: Being stochastically variable objects, an alert generated by an AGN should have flux from the source in both the reference and science stamps. Considering this feature alone, it is difficul. t to discriminate AGNs from other variable sources. Nevertheless, AGNs should lie at the centers of their host galaxies, or appear as (quasi-)stellar objects, in relatively lower stellar density fields. Thus, a change in flux will appear as a variable source, which may lie at the center of a galaxy, or even when the galaxy is not visible they tend to be in lower stellar density fields. In these cases, the alert is likely to be triggered by an AGN. In addition, AGNs are commonly found outside the Galactic plane.
    
+ Supernovae (SNe): An alert generated by a SN should appear as a change in flux where no unresolved sources were present. These transients tend to appear near their host galaxies, and their location should be consistent with the underlying host stellar population distribution (e.g., a SN will have a higher probability of arising from a location aligned with the disk than perpendicular to it). As such, most SN detections exhibit a visible host galaxy in both the science and reference stamps, with the flux from the SN arising only in the science and difference images. SN candidates tend to appear outside the Galactic plane due to occlusion.
    
+ Variable Stars (VS): The flux coming from variable stars usually appears in both the reference and science stamps. With ZTF's sensitivity, variable stars can be detected within the Milky Way or the Local Group, and thus the alert will typically not be associated with a visible host galaxy in the stamp, but rather with other point-like sources. In addition, such alerts will have a higher probability of residing at lower Galactic latitudes and in crowded fields with multiple point sources within the stamps, given the high concentration of stars in the disk and bulge of our Galaxy.

+ Asteroids: Alerts from moving Solar-system objects will appear only one time at a given position, and thus will show flux only in the science and difference images. Depending on their distance and speed, they may appear elongated in the direction of motion. In addition, such alerts should have a higher probability of residing near the ecliptic.
    
+ Bogus alerts: Camera and telescope optics effects, such as saturated pixels at the centers of bright sources, bad columns, hot pixels, astrometric misalignment in the subtraction to compute the difference image, unbaffled internal reflections, etc., can produce bogus alerts with no interesting real source. Bogus alerts are characterized by the presence of NaN pixels due to saturation, single or multiple bright pixels with little or no spatial extension (i.e., smaller than the telescope point spread function (PSF) and nightly seeing), or lines with high or low pixel values that extend over a large portion of the stamp (hot or cold columns/rows).

<img src="figures/ml/class_samples.png" width=90% />

Here we show examples of the five classes that are to be discriminated by using only the first detection. For each class, the triplet of images in each row are science, reference and difference images from left to right. Each row corresponds to a different candidate. The images are cropped at the center resulting in 21x21 images, and normalized. Further details about the pre-processing of the images is explained in Carrasco-Davis et al 2021.

### Convolutional neural network architecture: <a class="anchor" id="CNN"></a>

<table><tr><td><img src="figures/ml/cnn_architecture.png"></td><td><img src="colab/figures/ml/cnn_table.png"></td></tr></table>

The Figure on the left represent the stamp classifier. The box Convolutional Layers refers to those described in the Table at the right side, from the first convolutional layer to the last pooling layer. For each sample, the science, reference and difference images are concatenated in the channel dimension, obtaining an image input of dimension $21 \times 21 \times 3$. For each sample within the sampled batch, rotated versions are generated and fed to the CNN. After the first dense layer, the dense vectors of each the rotated version for the same objects are averaged. The metadata features pass through a batch normalization layer, the output of which is concatenated with the cyclic pooling output. Then, the concatenation goes through 2 fully connected layers, and finally a softmax function is applied to estimate the output probabilities. For the convolutional layers, the parameters shown in the table at the right are the filter dimensions and number of output channels. All the convolutional layers and fully connected layers have a Recified Linear Unit (ReLU) activation function (except for the last fully connected one that has a softmax output).

When the CNN model is trained using cross-entropy as the loss function to be minimized, the classification confidence of the model is very high, resulting in a distribution of output probabilities with saturated values of 0s and 1s without populating the values in between, even for wrong classifications. In this case there is no insight of certainty (relative probabilities between classes) of the prediction because most estimated probabilities for each class were either 0 or 1. In order to provide more granularity to the astronomers who revise SN candidates reported by the model to later request follow-up observing time, we added the entropy of the predicted probabilities of the models as a regularization term, to be maximized during training. By maximizing the entropy of the output probabilities, we penalize predictions with high confidence, in order to get better insight in cases where the stamps seem equally likely to belong to more than one class. The loss function $\mathcal{L}$ per sample is as follows:

$$    \mathcal{L} = \underbrace{-\sum_{c=1}^{N}y_{c}\log{(\hat{y}_{c})}}_{\text{cross-entropy}} + \underbrace{\beta \sum_{c=1}^{N}\hat{y}_{c}\log{(\hat{y}_{c})}}_{\text{entropy regularization}},$$
where $N$ is the number of classes, $y_{c}$ is the one-hot encoding label (a value of 1 in the corresponding index of class, and 0 for the rest)  indexed by $c$, $\hat{y}_{c}$ is the model prediction for class $c$, and $\beta$ controls the regularization term in the loss function.

### Clasification results: <a class="anchor" id="Results"></a>

<table><tr><td><img src="figures/ml/cm.png"></td><td><img src="colab/figures/ml/betas.png"></td></tr></table>

In the left image we show the confusion matrix in the test set for the proposed model. We use accuracy to compare models since the validation and test sets are balanced; achieving $0.95 \pm 0.005$ in the validation set and $0.941 \pm 0.004$ in the test set. To compute the confusion matrix we run five realizations of the mentioned model. With our five class model, we recover $87 \pm 1\%$ of the SNe, with only $5 \pm 2\%$ of false positives. By inspecting the predictions made by our model for each SN sample in the test set, we found that the results are in agreement with our initial expectations regarding the class discrimination described in the Training Data section. It is worth highlighting that the results of our model are achieved by using **the first alert only**.

In the right image, we show the probability distribution for each of the classes in the training set, for different values of the regularization constant $\beta=\{ 0, 0.5, 1.0 \}$. For the model without regularization ($\beta = 0$ shown on the top plot), the probability distribution saturates to 1 or 0. Increasing $\beta$ to 0.5 or 1.0 decreases the saturation and spreads the distribution of predictions made by the model (mid and bottom plots). In the case of $\beta=0$, the predictions are mostly saturated around 0 or 1 for the SN, VS, Asteroids and Bogus alert classes, creating difficulties to identify stamps that seem equally likely to belong to more than one class, because every sample is mapped to similar levels of high certainty. As the value of $\beta$ increases, the saturation of predicted values decreases, spreading the predicted probability distributions and emphasizing the different levels of certainty between predictions of different samples. The use of regularization to find noticeable differences in the predicted probabilities are helpful to experts for evaluating the output of the classifier, gaining better insight into how reliable the classifications are.

# ALeRCE frontend and Supernova Hunter <a class="anchor" id="Front"></a>

The [Supernova Hunter](https://snhunter.alerce.online/) is a visualization tool that allows the user to inspect SN candidates classified by the model in real time, in order to select good targets for follow-up observations.

<img src="figures/ml/sn_hunter.png" width=90% />

On the bottom left side, the location of each candidate in sky coordinates with respect to the Galactic plane and the ecliptic are depicted. On the bottom right side, a selection of the top candidates is listed, initially ordered by SN probability score from the stamp classifier. The list of candidates can be sorted by other parameters, and updated/refreshed to include newly ingested alerts. On the top left side, the SN candidate ID is shown as a clickable link to the [ALeRCE frontend](https://alerce.online/object/ZTF20abpvolc), with relevant metadata such as radec coordinates, magnitude, date, etc. At the bottom there are links to other sources of information, including ALeRCE, NED, TNS, and the Simbad Astronomical Database. In the middle of the figure there is a colored image from Aladin. On the top right side, the stamps of the first detection are shown, along with buttons for reporting the candidate as eventual bogus or as a possible SN.

<img src="figures/ml/alerce_frontend.png" width=90% />

When the candidate is inspected using the [ALeRCE frontend](https://alerce.online/object/ZTF20abpvolc), the full prediction of classes by the stamp classifier is shown at the bottom in addition to the full information of the light curve available, metadata, cross matches, etc. When the light curve of the object has six or more points in one of the bands, it is classified by the light curve classifier (Sánchez-Sáez et al. 2021), with the prediction also available in the ALeRCE frontend with a more complex taxonomy.

# Requirements <a class="anchor" id="req"></a>

In [ ]:
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
import ephem
import astropy.units as units
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import copy
import pickle
import matplotlib.cm as cm

# ALeRCE client <a class="anchor" id="client"></a>

Install and import the ALeRCE Python client with `pip install alerce`

In [ ]:
!pip install alerce

In [ ]:
from alerce.core import Alerce
client = Alerce()

### Plotting light curves <a class="anchor" id="plot_lc"></a>

We will create a simple function that plots the light curve given an object id (oid), a dataframe with detections and a dataframe with non detections. This function will be used to plot light curves of some objects predicted by the stamp classifier.

In [ ]:
def plotLC(oid, SN_det, SN_nondet):
    
    colors = {1: '#56E03A', 2: '#D42F4B'} 
    fig, ax = plt.subplots(figsize = (14, 7))
    fig.set_facecolor('white')
    labels = {1: 'g', 2: 'r'}
    markers = {1: 'o', 2: 's'}
    sizes = {1: 30, 2: 60}
    
    # loop the passbands
    for fid in [1, 2]:
        
        # plot detections if available
        mask = SN_det.fid == fid
        if np.sum(mask) > 0:
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            ax.errorbar(SN_det[mask].mjd, SN_det[mask].magpsf, 
                yerr = SN_det[mask].sigmapsf, c=colors[fid], label=labels[fid], marker=markers[fid])
        
        # plot non detections if available
        if len(SN_nondet) != 0:
            mask = (SN_nondet.fid == fid) & (SN_nondet.diffmaglim > -900)
            if np.sum(mask) > 0:     
                # non detections index is mjd
                ax.scatter(SN_nondet[mask].mjd, SN_nondet[mask].diffmaglim, c=colors[fid], alpha = 0.5,
                    marker='v', label="lim.mag. %s" % labels[fid], s=sizes[fid])

    ax.set_title(oid)
    ax.set_xlabel("MJD")
    ax.set_ylabel("Apparent magnitude")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

def getObjectData(oid, doLC=False, dostamp=False):

    results = {"oid": oid}
        
    # query detections
    SN_det = client.query_detections(oid, format='pandas')
    SN_det = SN_det.sort_values("mjd")
    results["lc_det"] = SN_det
        
    # query non detections
    SN_nondet = client.query_non_detections(oid, format='pandas')
    if len(SN_nondet)!=0:
        SN_nondet = SN_nondet.sort_values("mjd")
        results["lc_nondet"] = SN_nondet
    
    # plot the LC
    if doLC:
        plotLC(oid, SN_det, SN_nondet)
        
    # show the first image stamp
    if dostamp:
        candid = results["lc_det"].loc[results["lc_det"].has_stamp].candid.min()
        stamps = client.get_stamps(oid, candid)
        science, ref, difference = stamps[0].data, stamps[1].data, stamps[2].data
        fig, ax = plt.subplots(ncols=3, figsize=(12, 6))
        fig.set_facecolor('white')
        for idx, im in enumerate([np.arcsinh(science), np.arcsinh(ref), difference]):
            ax[idx].imshow(im, cmap='viridis') # Log scale for visualization
            ax[idx].axes.get_xaxis().set_visible(False)
            ax[idx].axes.get_yaxis().set_visible(False)
        ax[0].set_title("oid: %s, candid: %s (science, reference and difference)" % (oid, candid), loc='left')
        fig.subplots_adjust(wspace = 0, hspace = 0)
        
    # return data
    return results

# Classified Alerts <a class="anchor" id="get_alerts"></a>

### Get objects per class
Here, we build a function that uses the ALeRCE client to query objects according to the predictions of the stamp classifier. In order to select the classifier, we define the query in terms of the classifier name, corresponding class and their minimum probability, the classes are the following strings:
+ "AGN": Active Galactic Nuclei
+ "SN": Supernova
+ "VS": Variable star
+ "Asteroid"
+ "Bogus"

Using the ALeRCE client we can select the min and max number of observations per object, also sort them by firstmjd, number of observations (ndet), probability of the selected class, etc. 

In [ ]:
def get_objects_per_class(classearly="SN", pclassearly=0.8, n_objects=100):

    min_firstmjd = Time("2022-06-01T00:00:00", format="isot", scale="utc").mjd

    objects = client.query_objects(classifier="stamp_classifier",
                                   class_name=classearly,
                                   #classifier_version="stamp_classifier_1.0.4",
                                   probability=pclassearly,
                                   ranking=1,
                                   #ndet=[1, 50],
                                   #order_by="probability",
                                   #order_mode="DESC",
                                   count=False,
                                   first_mjd=[min_firstmjd, None],
                                   page_size=n_objects, 
                                   format='pandas')
    print(objects.shape)
    objects.head()
    objects.set_index("oid", inplace=True)
    objects.sort_values(by="ndet", inplace=True, ascending=False)
    return objects

The previous function returns a pandas dataframe with a list of objects by their ZTF oid as index, and relevant information in their columns as we will see next.
Now, let's query some objects for each of the classes available from the stamp classifier. Here, we are querying a few thousands of objects from the ALeRCE database. We will also sort the classes by ascending number of observations in the cases of bogus and asteroids in order to get a clean example. AGNs, VS and SNe are sorted in descending order of number of observations to better appreciate the light curves for each of these classes.

**This query will take a few seconds**

In [ ]:
n_objects = 2000 # Objects per class to query
early_classes = ["AGN", "SN", "VS", "asteroid", "bogus"] # Class identifiers to query objects
objects = {} # Initialize dictionary to use the results per class
min_nobs = 1
for i, cl in enumerate(early_classes):
    print(cl)
    objects[cl] = get_objects_per_class(classearly=cl, n_objects=n_objects)
    if i==0:
        print("Result of a query using the ALeRCE client")
        display(objects[cl].head())
        print("Columns available", objects[cl].columns)

# Plot some examples per class <a class="anchor" id="examples"></a>

Now we have a dataframe per class resulting from our queries. The following function is just to simplify the inspection of candidates by printing the URL to the ALeRCE frontend, and call the getObjectData function done previously with the flags doLC and dostamp True to plot everything.

In [ ]:
def plot_some_LC(df, n_examples=3):
    for i in range(n_examples):
        object_id = df.iloc[i].name
        print(f"https://alerce.online/object/{object_id}")
        getObjectData(object_id, doLC=True, dostamp=True)
        plt.show()

In [ ]:
for cl, df in objects.items():
    print("######### "+ cl + " class #########")
    print(cl)
    display(df)
    if cl in ["asteroid", "bogus"]: # Just to have well classified samples to show
        df.sort_values(by="ndet", inplace=True)
    plot_some_LC(df, n_examples=2)

The distribution of the number of detections per class can give us insights on how well the classifier is working. For instance, for AGNs, SNe, and VS, we expect to see more than one detection for these classes. The objects belonging to these classes and have one detection could be new objects or the ones visited only once, short transients, or errors in the classification. For asteroids and bogus class, we expect to have objects with one detection only.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(20, 8))
fig.set_facecolor('white')
bins = np.arange(0, 40)
ax = ax.flatten()
fontsize = 16

# Concatenated dataframe with all the classes
h, _ = np.histogram(pd.concat(list(objects.values()), axis=0).ndet, bins=bins, density=True)
ax[0].bar(bins[:-1], h, label="all_data")
ax[0].set_title("all data", fontsize=fontsize)
ax[0].tick_params(axis='both', which='major', labelsize=fontsize)
ax[0].set_ylim([0, 1])

for i, (cl, df) in enumerate(objects.items()):
    # Histogram per class
    h, bins = np.histogram(df.ndet, bins=bins, density=True)
    ax[i+1].bar(bins[:-1], h, label=cl)
    ax[i+1].set_title(cl, fontsize=fontsize)
    ax[i+1].tick_params(axis='both', which='major', labelsize=fontsize)
    ax[i+1].set_ylim([0, 1])
    
plt.show()

In our small sample we can see that most of the asteroids and bogus have only one detection, while AGNs, SNe and VS have a long tail of objects with more than one detection.

# Get a sequence of stamps <a class="anchor" id="stamp_series"></a>

In this section, we will implement a function to get a series of stamps from the alerts availables for a given object. The stamp classifier uses the first alert only, but in ALeRCE we are working to extend the stamp classifier to use a multi-stamp approach considering band g and r simultaneously. One example of this kind of classifier is the one presented in [Carrasco-Davis et al. 2019](https://iopscience.iop.org/article/10.1088/1538-3873/aaef12), where the stamps are processed by a convolutional neural network, to then use a recurrent neural network that can store previous inputs of the sequence of stamps. Then, in case you want to experiment with a sequence of stamps, here is an example:
+ We first get the detections of an object using get_detection of the ALeRCE client
+ Then we sort the values by mjd
+ The resulting dataframe uses the candidate id as index, then we use the get_stamp function for each candidate to obtain their respective stamp
+ We finally plot the stamps and return a list of stamps

In [ ]:
def get_stamp_series(oid, n_stamps=5, show_stamps=False, show_LC=False):
    band_labels = {1: 'g', 2: 'r'}
    band_cmap = {"g": "viridis", "r": "inferno"}
    object_det = client.query_detections(oid, format='pandas') # Get object detections
    candids = object_det.loc[object_det.has_stamp]
    object_det = object_det.iloc[:n_stamps] # Use n stamps only
    candids = object_det.candid # Obtain the candidate id to use it in the get_stamp function
    stamps = []
    for c in candids:
        candid_stamps = client.get_stamps(oid, candid=c)
        stamps.append(candid_stamps) # Save the stamps in a list to return
    
    if show_LC:
        getObjectData(oid, doLC=True, dostamp=False) # Plot the light curve of the object
        
    if show_stamps:
        fig, ax = plt.subplots(3, n_stamps, figsize=(n_stamps*3, 10))
        fig.set_facecolor('white')
        for i, s in enumerate(stamps):
            corresponding_band = band_labels[object_det.iloc[i].fid] # Check band
            cmap = band_cmap[corresponding_band] # use a different cmap for each band
            # The get_stamp function returns a list with three images 
            # (science, reference and difference) in hdu format. Using .data we get the numpy array.
            science, ref, difference = s[0].data, s[1].data, s[2].data 
            ax[0, i].imshow(np.arcsinh(science), cmap=cmap) # Log scale for visualization
            ax[1, i].imshow(np.arcsinh(ref), cmap=cmap)
            ax[2, i].imshow(difference, cmap=cmap)
            title = "MJD " + str(np.round(object_det.iloc[i].mjd, 2)) + ", band: "+corresponding_band
            ax[0, i].set_title(title)
        ax[0, 0].set_ylabel("Science")
        ax[1, 0].set_ylabel("Reference")
        ax[2, 0].set_ylabel("Difference")
        plt.show()
    return object_det, stamps

Let's use [ZTF19acftude](https://alerce.online/object/ZTF19acftude) since it has a very nice looking SN light curve and host galaxy.

In [ ]:
detections, stamps = get_stamp_series(oid="ZTF19acftude", show_stamps=True, show_LC=True)

The detections dataframe contains important information regarding the specific properties measured at a given time for the same object. For instance, the measuring time in mjd, mean and std of the position in radec, the magnitude, etc.

In [ ]:
detections

In [ ]:
detections.columns

# Get spatial distribution of predictions <a class="anchor" id="spatial_distribution"></a>

One issue when training models in astronomy is the possible biases found in the training set. One of the problems we had at the beginning was the abundance of extragalactic objects predicted near the galactic plane. As mentioned in the Introduction, SNe and AGNs are less likely to be found near the galactic plane due to occlusion. We fixed this problem by adding some features to the CNN at the fully connected layers, one of the most important are the ecliptic and galactic coordinates, so let's see how they look like per class.

### Compute galactic and ecliptic coordinates
We computed the coordinates from the meanra and mean dec using the ephem python package. Here (https://rhodesmill.org/pyephem/coordinates.html) there is a more detailed tutorial of how to use this transformation between coordinates. Now, we will take our dataframes and add ecliptic and galactic latitude and longitude as columns.

In [ ]:
def ecliptic_coordinates(df):
    # Define the lambda function to be applied to each row of the dataframe
    ecl = df.apply(lambda row: ephem.Ecliptic(ephem.Equatorial('%s' % (row.meanra / 15.),
                                                               '%s' % row.meandec, epoch=ephem.J2000)), axis=1)
    
    # Apply the function to each row, the result is append as a new column
    df["ecl_lat"] = ecl.apply(lambda row: np.rad2deg(row.lat))
    df["ecl_long"] = ecl.apply(lambda row: np.rad2deg(row.long))
    
    # Return the updated dataframe
    return df


def galactic_coordinates(df):
    # Define the lambda function to be applied to each row of the dataframe
    gal = df.apply(lambda row: ephem.Galactic(ephem.Equatorial('%s' % (row.meanra / 15.),
                                                               '%s' % row.meandec, epoch=ephem.J2000)), axis=1)
    # Apply the function to each row, the result is append as a new column
    df["gal_lat"] = gal.apply(lambda row: np.rad2deg(row.lat))
    df["gal_long"] = gal.apply(lambda row: np.rad2deg(row.long))
    
    # Return the updated dataframe
    return df

# For each of our dataframe corresponding to one of the classes of the stamp classifier
# we add the ecliptic and galactic coordinates
for cl, df in objects.items():
    ecl_df = ecliptic_coordinates(df)
    gal_df = galactic_coordinates(ecl_df)
    objects[cl] = gal_df

In [ ]:
def plot_spatial_distribution(df, ax, index=0, cmin_val=0.1, vmax_val=5, titles="", fontsize=18, x_label=None, y_label=None):
    dims = ['gal_long', 'gal_lat'] # Let's use our new column to plot the objects
    # This is a 2D histogram of the positions of objects for each of the classes
    _, _, _, im = ax.hist2d(df[dims].values[:, 0], df[dims].values[:, 1],
                            (300, 300), cmap="viridis", vmax=vmax_val, cmin=cmin_val)
    ax.tick_params(axis='both', labelsize=fontsize-2)
    ax.set_xlim([0, 360])
    ax.set_ylim([-80, 80])
    cbar = plt.colorbar(im,ax=ax, pad=0)
    cbar.ax.tick_params(labelsize=fontsize-4)
    
    ecliptic_lat = np.zeros(500)
    ecliptic_longi = np.linspace(0, 360, num=500)
    # This auxiliary function computes the ecliptic plane by converting
    # the ecliptic latitude 0 to galactic coordinates
    # We should find many asteroids near the ecliptic
    ecliptic = SkyCoord(ecliptic_longi, ecliptic_lat, unit='deg', frame='barycentrictrueecliptic')
    galact_long, galact_lat = ecliptic.galactic.l.deg, ecliptic.galactic.b.deg
    
    ax.set_title(titles, fontsize=fontsize)
    ax.plot(galact_long, galact_lat, "ok", markersize=6)
    ax.plot(galact_long, galact_lat, "oy", markersize=4)
    if not x_label is None:
        ax.set_xlabel(x_label, fontsize=fontsize)
    if not y_label is None:
        ax.set_ylabel(y_label, fontsize=fontsize)
    
    return ax

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(3*10, 2*6))
fig.set_facecolor('white')
plt.subplots_adjust(wspace=0.0, hspace=0.15)
ax = ax.flatten() # A trick to iterate over the axis of a subplot array
titles = ["all objects", ] + list(objects.keys())
# This is just to control the saturation of the histograms, the larger
# the number, the larger the amount of objects needed in the same location to increase the color scale
max_count_per_class = dict(zip(titles, [20, 5, 5, 10, 5, 5])) 

for i in range(6):
    x_label = None
    y_label = None
    if i == 0:
        df = pd.concat(list(objects.values()), axis=0)
    else:
        df = list(objects.values())[i-1]
    
    if i in [0, 3]:
        y_label = "Galactic Latitude"
    if i in [3, 4, 5]:
        x_label = "Galactic Longitude"
    ax[i] = plot_spatial_distribution(df, ax[i], titles=titles[i], vmax_val=max_count_per_class[titles[i]], x_label=x_label, y_label=y_label)
plt.show()

Here us the spatial distribution for the unlabeled data, and distribution of predictions per class. The colorbar indicates the density of points. The ecliptic is shown with a yellow line with black edges. The distributions are shown as a 2d histogram of density of alerts. Extragalactic sources (SNe and AGNs) are found outside the Galactic plane. On the contrary, VS are concentrated in the Galactic plane. Asteroids are near the ecliptic.

### Probabilities per class as a function of the position

Even though most of the objects per class lie in a reasonable position according to their nature, there are some errors or contamination between classes. It is interesting to see how certain is the stamp classifier for each class depending on their position, so let’s plot the probability of each object belonging to each class in the same galactic coordinates projection. We only have to modify our previous “plot_spatial_distribution” function to make a scatter plot instead of the 2d histogram. We will fill each scatter dot with a colormap and size that reflects their probability.

In [ ]:
def plot_spatial_probabilities(df, ax, index=0, cmin_val=0.1, vmax_val=5, titles="", fontsize=18, x_label=None, y_label=None):
    # We sort the rows of the dataframe first in order to reduces biases when "painting" the plot with the scatter
    df = df.sample(frac=1) 
    dims = ['gal_long', 'gal_lat'] # Let's use our new column to plot the objects
    # We will use the normalized probabilities (between 0 and 1) to scale the size of each dot according 
    # the their probability of belonging to the respective class
    normalized_probabilities_per_class = df["probability"].values - np.amin(df["probability"].values)
    normalized_probabilities_per_class = normalized_probabilities_per_class/np.amax(normalized_probabilities_per_class)
    # Variable size dots for the scatter plot
    variable_sizes = 10 + normalized_probabilities_per_class*100
    color_map = plt.cm.get_cmap('viridis')
    im = ax.scatter(df[dims].values[:, 0], df[dims].values[:, 1], c=df["probability"].values, cmap=color_map,
                   s=variable_sizes, alpha=0.7)
    ax.tick_params(axis='both', labelsize=fontsize-2)
    ax.set_xlim([0, 360])
    ax.set_ylim([-80, 80])
    cbar = plt.colorbar(im,ax=ax, pad=0)
    cbar.ax.tick_params(labelsize=fontsize-4)
    
    ecliptic_lat = np.linspace(0, 0, num=500)
    ecliptic_longi = np.linspace(0, 360, num=500)
    # This auxiliary function computes the ecliptic plane by converting
    # the ecliptic latitude 0 to galactic coordinates
    ecliptic = SkyCoord(ecliptic_longi, ecliptic_lat, unit='deg', frame='barycentrictrueecliptic')
    galact_long, galact_lat = ecliptic.galactic.l.deg, ecliptic.galactic.b.deg
    
    ax.set_title(titles, fontsize=fontsize)
    ax.plot(galact_long, galact_lat, "ok", markersize=6)
    ax.plot(galact_long, galact_lat, "oy", markersize=4)
    
    if not x_label is None:
        ax.set_xlabel(x_label, fontsize=fontsize)
    if not y_label is None:
        ax.set_ylabel(y_label, fontsize=fontsize)
    
    return ax

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(3*10, 2*6))
fig.set_facecolor('white')
plt.subplots_adjust(wspace=0.0, hspace=0.15)
ax = ax.flatten() # A trick to iterate over the axis of a subplot array
titles = ["all objects", ] + list(objects.keys())

for i in range(6):
    x_label = None
    y_label = None
    if i == 0:
        df = pd.concat(list(objects.values()), axis=0)
    else:
        df = list(objects.values())[i-1]
    
    if i in [0, 3]:
        y_label = "Galactic Latitude"
    if i in [3, 4, 5]:
        x_label = "Galactic Longitude"
    
    ax[i] = plot_spatial_probabilities(df, ax[i], titles=titles[i], vmax_val=max_count_per_class[titles[i]], x_label=x_label, y_label=y_label)
plt.show()

For the SN class, the stamp classifier seems to be more confident about the prediction further from the ecliptic, there is a large blob of green-yellow dots at around (40, 120), while darker blobs near the ecliptic, meaning less confidence in the prediction. We can also see a greener blob near the center of the galaxy in the case of variable stars. In addition, it is also important to know the relative confidence between classes, some classes are harder to classify than others as you can see in the “all objects” scatter. Now, we will plot the distribution of probabilities per class.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
fig.set_facecolor('white')
for i in range(6):
    if i == 0:
        df = pd.concat(list(objects.values()), axis=0)
        h, bins = np.histogram(df.probability.values, bins=10, density=True)
    else:
        df = list(objects.values())[i-1]
        h, bins = np.histogram(df.probability.values, bins=bins, density=True)
    ax.plot(bins[:-1], h, label=titles[i])
    
plt.legend()
ax.set_xlabel("Class probability", fontsize=16)
ax.set_ylabel("Freq", fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=16)
ax.set_yscale('log')
plt.show()